In [1]:
import pandas as pd
import time
import re
import os
from llama_cpp import Llama
from fuzzywuzzy import fuzz
import json
from Levenshtein import ratio, distance
from pathlib import Path
import random
import xlsxwriter
from tqdm import tqdm

In [2]:
output_dir_stats = Path("TableJoiner")
output_dir_stats.mkdir(parents=True, exist_ok=True)

input_file = './Merged_tables/merged_global.xlsx'
llm =Llama(model_path="./IMPORTS/mistral-7b-instruct-v0.1.Q4_K_M.gguf", n_ctx=2048, verbose=False)

llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


In [3]:
def normalize_address(address):
    address = str(address).lower()
    address = re.sub(r'[^а-я0-9\s]', ' ', address)
    address = re.sub(r'\s+', ' ', address).strip()
    address = address.replace('улица ', 'ул ')
    address = address.replace('у ', 'ул ')
    address = address.replace('проспект ', 'пр ')
    address = address.replace('набережная ', 'наб ')
    address = address.replace('переулок ', 'пер ')
    address = address.replace('пр кт ', 'пр ')
    address = address.replace('просп ', 'пр ')
    address = address.replace('корпус ', 'к ')
    address = address.replace('шоссе ', 'ш ')
    address = address.replace('шосе ', 'ш ')
    address = address.replace('корп ', 'к ')
    address = address.replace('строение ', 'стр ')
    address = address.replace('литера ', 'л ')
    address = address.replace('литер ', 'л ')
    address = address.replace('город ', 'г ')
    address = address.replace('область ', 'обл ')
    address = address.replace('дом ', 'д ')    
    return address

In [4]:
def extract_numbers(address):
    return re.findall(r'\d+[а-яА-Я]*', address.lower())

def compare_number_sequences(nums1, nums2, mismatches = 0):
    nums1.sort()
    nums2.sort()
    len_diff = abs(len(nums1) - len(nums2))
    if len_diff > 1:
        return False
    if len_diff == 1:
        longer, shorter = (nums1, nums2) if len(nums1) > len(nums2) else (nums2, nums1)
        for i in range(len(longer)):
            temp = longer[:i] + longer[i+1:]
            if compare_number_sequences(temp, shorter, 1):
                return True
        return False    
    
    for n1, n2 in zip(nums1, nums2):
        if n1 != n2:
            mismatches += distance(str(n1), str(n2))
            if mismatches >= 2:
                return False
    return True

# nums1 = extract_numbers("спб пркт просвещения д 102 корп а")
# nums2 = extract_numbers("спб пркт просвещения д 87 корп 2")
# print(compare_number_sequences(nums1, nums2))

In [5]:
def extract_street_name(address):
    # Сначала пробуем найти улицу перед "д"
    address = re.sub(r'([а-яё])(\d)', r'\1 \2', address, flags=re.IGNORECASE)
    address = re.sub(r'([а-яёa-z0-9])[,.;]+([а-яёa-z0-9])', r'\1 \2', address, flags=re.IGNORECASE)
    
    # Удаляем оставшиеся знаки препинания по краям
    address = re.sub(r'^[,.;]+|[,.;]+$', '', address)

    match = re.search(r'(.+?)\s+д\b', address, re.IGNORECASE)
    if match:
        before_house = match.group(1).strip()
        parts = before_house.split()
        if len(parts) >= 2:
            last_two = parts[-2:]
            street = [word for word in last_two if len(word) >= 4]
            if len(street) >= 4:
                return street
            else:
                return max(last_two, key=len)
        
        if len(parts) >= 1 and len(parts[-1]) >= 4:
            return parts[-1]
    
    patterns = [
        r'(?:ул|пр|пер|ш|наб)[\s\.]+([^,\d]+?)(?:\s+(?:|д|к|стр|ул|пр|пер|ш|наб))?', # между улицей и домом
        r'(?:ул|пр|пер|ш|наб)[\s\.]+([^,\d]+)',  # после ключевого слова
        r'([^,\d]+?)\s+(?:ул|пр|пер|ш|наб)[\s\.]*$',  # перед ключевым словом
    ]

    def find_max_segment(text):
        words = [word for word in re.findall(r'[а-яёa-z-]+', address, re.IGNORECASE) if len(word) >= 4]
        return ' '.join(words) if words else None
    
    for pattern in patterns:
        match = re.search(pattern, address, re.IGNORECASE)
        if match:
            street = match.group(1).strip()
            street = re.sub(r'\s*(?:д |к |стр |ул |пр |пер |ш |наб ).*$', '', street, flags=re.IGNORECASE)
            if street:
                res = find_max_segment(street)
                if res is not None and len(res) >= 4:  # Проверяем минимальную длину названия
                    return res
    
    words = re.findall(r'[а-яёa-z-]+|\d+', address, re.IGNORECASE)
    
    last_number_pos = -1
    for i in range(len(words)-1, -1, -1):
        if words[i].isdigit():
            last_number_pos = i
            break
    
    if last_number_pos > 0:
        result_words = []
        for word in reversed(words[:last_number_pos]):
            if len(word) >= 4:
                result_words.insert(0, word)
            elif not result_words:
                continue
            else:
                break
        if result_words:
            return ' '.join(result_words)
    
    return None

In [6]:
def check_equal(llm, address1, address2):
    address1 = normalize_address(address1)
    address2 = normalize_address(address2)
    
    if address1 == address2:
        return "да",1
    
    nums1 = extract_numbers(address1)
    nums2 = extract_numbers(address2)
#   Совпадающие номера
    if not compare_number_sequences(nums1,nums2):
        return "нет",1
    
#   Совпадающие улицы
    street1 = extract_street_name(address1)
    street2 = extract_street_name(address2)
    
    if street1 and street2:
        if distance(street1, street2) <= 2:
            return "да",1 
        elif distance(street1, street2) > 4:
            return "нет",1   
    
#    Расстояние Левенштейна
    similarity = ratio(address1, address2)
    if similarity >= 0.95:
        return "да",1
    elif similarity <= 0.80:
        return "нет",1
    
#     fuzzy не подходит т.к. он не находит опечатки

#     similarity = fuzz.token_sort_ratio(normalized_addr1, normalized_addr2)
#     return similarity >= threshold
    
    prompt = f"""
    Анализируй следующие два адреса максимально строго:
    1. {address1}
    2. {address2}

    Ответь "Да" ТОЛЬКО если:
    - Это точно один и тот же адрес (совпадают все ключевые элементы)
    - Лишь незначительные различия в написании (опечатки, сокращения)
    - Разные варианты написания одного места

    Во всех остальных случаях отвечай "Нет". Особенно если:
    - Есть различия в номерах домов/корпусов
    - Названия отличаются более чем на 30%
    - Присутствуют разные географические указатели
    - Есть сомнения в идентичности

    Твой ответ (только "Да" или "Нет"):
    """
    try:
        response = llm.create_chat_completion(
                messages=[{"role": "user", "content": prompt}]
            )
        result = response["choices"][0]["message"]["content"].strip().lower()
        if "да" in result:
            return "да",0
        else:
            return "нет",0
    except Exception as e:
        print(f"Ошибка при запросе к LLM: {e}")
        return "нет", 0

In [7]:
def check_equal_llm(llm, address1, address2):
    prompt = f"""
    Анализируй следующие два адреса максимально строго:
    1. {address1}
    2. {address2}

    Ответь "Да" ТОЛЬКО если:
    - Это точно один и тот же адрес (совпадают все ключевые элементы)
    - Лишь незначительные различия в написании (опечатки, сокращения)
    - Разные варианты написания одного места

    Во всех остальных случаях отвечай "Нет". Особенно если:
    - Есть различия в номерах домов/корпусов
    - Названия отличаются более чем на 30%
    - Присутствуют разные географические указатели
    - Есть сомнения в идентичности

    Твой ответ (только "Да" или "Нет"):
    """
    try:
        response = llm.create_chat_completion(
                messages=[{"role": "user", "content": prompt}]
            )
        result = response["choices"][0]["message"]["content"].strip().lower()
        if "да" in result:
            return "да"
        else:
            return "нет"
    except Exception as e:
        print(f"Ошибка при запросе к LLM: {e}")
        return "нет"

In [8]:
def process_excel(input_path, output_path):
    start_time = time.time()
    df = pd.read_excel(input_path)
    
    standard_third_val_headers = [
        "Address1",
        "Address2",
        "Pred_type",
        "Type",
    ]
   
    test_third_val_heuristic = pd.DataFrame(columns=standard_third_val_headers)
    test_third_val_llm = pd.DataFrame(columns=standard_third_val_headers)
    
    df['Информация'] = df.apply(lambda row: {row['Страховая компания']: [row['Виды помощи'], row['Тип доступа']]}, axis=1)
    intermediate_df = df[['Адрес', 'Название поликлиники', 'Информация']].copy()
    
    processed_data = []
    unique_addresses = set()
    
    for i, row in tqdm(intermediate_df.iterrows(), total=len(intermediate_df), desc="Обработка адресов"):
        
        current_address = row['Адрес']
        current_name = row['Название поликлиники']
        combined_info = row['Информация'].copy()  
        
        for j, other_row in intermediate_df.iterrows():
            if j <= i:
                continue
                
            other_address = other_row['Адрес']
            info, flag = check_equal(llm,current_address, other_address)
            val_row = {"Address1": current_address, "Address2": other_address, "Pred_type": info}

            if flag == 1:
                if (i+j)%1000 == 0:
                    test_third_val_heuristic = pd.concat([test_third_val_heuristic, pd.DataFrame([val_row])], ignore_index=True)
            else:
                test_third_val_llm = pd.concat([test_third_val_llm, pd.DataFrame([val_row])], ignore_index=True)
            
            if info == "да":
                combined_info.update(other_row['Информация'])
                if len(other_row['Название поликлиники']) > len(current_name):
                    current_name = other_row['Название поликлиники']

        if current_address in unique_addresses:
            continue

        processed_data.append({
            'Адрес': current_address,
            'Название поликлиники': current_name,
            'Информация': combined_info
        })
        
        unique_addresses.add(current_address)

    
    result_df = pd.DataFrame(processed_data)
    
    output_dir_stats = Path("Stats")
    output_dir_stats.mkdir(parents=True, exist_ok=True)
    
    random.seed(42)
    sampled_rows = test_third_val_heuristic.sample(n=min(100, len(test_third_val_heuristic)), random_state=42)

    test_bonus_third_val_llm = test_third_val_llm
    for _, row in sampled_rows.iterrows():
        new_row = {
            "Address1": row["Address1"], 
            "Address2": row["Address2"], 
            "Pred_type": check_equal_llm(llm, row["Address1"],row["Address2"])
        }
        test_bonus_third_val_llm = pd.concat([test_bonus_third_val_llm, pd.DataFrame([new_row])], ignore_index=True)                            

    
    test_third_val_heuristic.to_excel(output_dir_stats / f"test_third_val_heuristic.xlsx", index=False)
    test_third_val_llm.to_excel(output_dir_stats / f"test_third_val_llm.xlsx", index=False)
    test_bonus_third_val_llm.to_excel(output_dir_stats / f"test_bonus_third_val_llm.xlsx", index=False)
    
    
    elapsed_time = time.time() - start_time
    print(f"Обработка завершена.")
    print(f"Затраченное время: {elapsed_time:.2f} сек")
    return result_df

In [9]:
def make_final_excel_file(df, output_file):
    writer = pd.ExcelWriter(
        output_file, 
        engine='xlsxwriter',
        engine_kwargs={'options': {'nan_inf_to_errors': True,'strings_to_urls': False}}
    )
    workbook = writer.book
    worksheet = workbook.add_worksheet()

    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'vcenter',
        'align': 'center',
        'fg_color': '#D7E4BC',
        'border': 1
    })
    subheader_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'vcenter',
        'align': 'center',
        'fg_color': '#D7E4BC',
        'border': 1
    })
    cell_format = workbook.add_format({
        'text_wrap': True,
        'valign': 'top',
        'border': 1
    })

    worksheet.merge_range('A1:A2', 'Адрес', header_format)
    worksheet.merge_range('B1:B2', 'Название поликлиники', header_format)

    unique_companies = set()
    for _, row in df.iterrows():
        unique_companies.update(row['Информация'].keys())
    unique_companies = sorted(list(unique_companies))

    col_offset = 2
    for company in unique_companies:
        worksheet.merge_range(0, col_offset, 0, col_offset + 1, company, header_format)
        worksheet.write(1, col_offset, 'Виды помощи', subheader_format)
        worksheet.write(1, col_offset + 1, 'Тип доступа', subheader_format)
        col_offset += 2

    row_num = 2
    for _, row_data in df.iterrows():
        worksheet.write(row_num, 0, row_data['Адрес'], cell_format)
        worksheet.write(row_num, 1, row_data['Название поликлиники'], cell_format)
        
        col_offset = 2
        for company in unique_companies:
            info = row_data['Информация'].get(company)
            if info:
                info0 = str(info[0]) if pd.notna(info[0]) else ""
                info1 = str(info[1]) if pd.notna(info[1]) else ""
                
                worksheet.write(row_num, col_offset, info0, cell_format)
                worksheet.write(row_num, col_offset + 1, info1, cell_format)
            else:
                worksheet.write(row_num, col_offset, '', cell_format)
                worksheet.write(row_num, col_offset + 1, '', cell_format)
            col_offset += 2
        row_num += 1

    worksheet.set_column('A:A', 30)
    worksheet.set_column('B:B', 40)
    for i in range(len(unique_companies) * 2):
        worksheet.set_column(2 + i, 2 + i, 20)

    workbook.close()
    print(f"Результат сохранен в {output_file}")


In [10]:
df = process_excel(input_file, './Joined_raw_llm.xlsx')

Обработка адресов: 100%|██████████████████| 7372/7372 [1:31:35<00:00,  1.34it/s]


Обработка завершена.
Затраченное время: 6643.38 сек


In [11]:
make_final_excel_file(df, './TableJoiner/FinalTable.xlsx')

Результат сохранен в ./TableJoiner/FinalTable.xlsx
